In [7]:
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np


In [4]:
# Load the embeddings file
embeddings_path = '/home/minhyekj/xraydata/chest-xray14_embeddings.npy'
embeddings = np.load(embeddings_path)

# Display the shape of the loaded embeddings
embeddings.shape

(9600, 512)

In [8]:
# Load the CSV file
data_entry_path = '/home/minhyekj/xraydata/Data_Entry_2017.csv'
data_entry = pd.read_csv(data_entry_path)

# Display the first few rows of the dataframe
data_entry.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [10]:
# Load the list of selected images
selected_images_path = '/home/minhyekj/xraydata/selected_png_list.txt'
with open(selected_images_path, 'r') as file:
    selected_images = file.read().splitlines()

# Display the first few selected image names
selected_images[:5]

['/home/younjoon/data/dataset/chestx-ray14/images_003/images/00004118_002.png',
 '/home/younjoon/data/dataset/chestx-ray14/images_003/images/00005241_000.png',
 '/home/younjoon/data/dataset/chestx-ray14/images_003/images/00005681_020.png',
 '/home/younjoon/data/dataset/chestx-ray14/images_003/images/00006469_000.png',
 '/home/younjoon/data/dataset/chestx-ray14/images_003/images/00005921_000.png']

In [11]:
# Extract the image file names from the full paths in the selected image list
selected_image_filenames = [path.split('/')[-1] for path in selected_images]

# Create a mapping of image filenames to their labels from the data entry dataframe
image_label_map = dict(zip(data_entry['Image Index'], data_entry['Finding Labels']))

# Extract labels for the selected images
selected_labels = [image_label_map[filename] for filename in selected_image_filenames if filename in image_label_map]

# Display the first few labels for the selected images
selected_labels[:5]

['Edema|Effusion|Infiltration|Pleural_Thickening|Pneumonia',
 'No Finding',
 'Cardiomegaly|Consolidation|Emphysema|Pneumothorax',
 'Cardiomegaly',
 'No Finding']

In [12]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np

# 가정: selected_images, data_entry가 이미 정의되어 있다.
# selected_images = list_of_full_paths
# data_entry = pd.DataFrame with columns ['Image Index', 'Finding Labels']

# Extract the image file names from the full paths in the selected image list
selected_image_filenames = [path.split('/')[-1] for path in selected_images]

# Create a mapping of image filenames to their labels from the data entry dataframe
image_label_map = dict(zip(data_entry['Image Index'], data_entry['Finding Labels']))

# Extract labels for the selected images
selected_labels = [image_label_map[filename] for filename in selected_image_filenames if filename in image_label_map]

# 데이터를 텐서로 변환, 가정: embeddings는 이미 준비된 임베딩 데이터
X = torch.tensor(embeddings, dtype=torch.float32)  # embeddings should be loaded or defined somewhere
y = torch.tensor([image_label_map[filename] for filename in selected_image_filenames if filename in image_label_map], dtype=torch.long)

# 데이터를 학습용과 테스트용으로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# DataLoader 설정
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 간단한 분류기 정의
class Classifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

# 모델 초기화
model = Classifier(input_dim=2048, num_classes=len(torch.unique(y)))  # ResNet의 특징 크기와 클래스 수
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    total_loss = 0
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

# 테스트 함수
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# 훈련 및 테스트
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer)
    test_accuracy = test(model, test_loader)
    print(f'Epoch: {epoch+1}, Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')


ValueError: too many dimensions 'str'

In [13]:
import numpy as np
import pandas as pd

# Load embeddings
embeddings_path = '/home/minhyekj/xraydata/chest-xray14_embeddings.npy'
embeddings = np.load(embeddings_path)

# Load CSV file for labels
data_entry_path = '/home/minhyekj/xraydata/Data_Entry_2017.csv'
data_entry = pd.read_csv(data_entry_path)

# Load the list of selected images
selected_images_path = '/home/minhyekj/xraydata/selected_png_list.txt'
with open(selected_images_path, 'r') as file:
    selected_images = file.read().splitlines()

# Extract the image file names from the full paths in the selected image list
selected_image_filenames = [path.split('/')[-1] for path in selected_images]

# Create a mapping of image filenames to their labels from the data entry dataframe
image_label_map = dict(zip(data_entry['Image Index'], data_entry['Finding Labels']))

# Extract labels for the selected images
selected_labels = [image_label_map[filename] for filename in selected_image_filenames if filename in image_label_map]

# Check the first few entries
selected_image_filenames[:5], selected_labels[:5]

(['00004118_002.png',
  '00005241_000.png',
  '00005681_020.png',
  '00006469_000.png',
  '00005921_000.png'],
 ['Edema|Effusion|Infiltration|Pleural_Thickening|Pneumonia',
  'No Finding',
  'Cardiomegaly|Consolidation|Emphysema|Pneumothorax',
  'Cardiomegaly',
  'No Finding'])

In [16]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

# Split labels into list of labels
split_labels = [label.split('|') for label in selected_labels]

# Binarize labels
mlb = MultiLabelBinarizer()
binary_labels = mlb.fit_transform(split_labels)

# Split the embeddings and binary labels into training and test sets
X_train, X_test, y_train, y_test = train_test_split(embeddings[:len(binary_labels)], binary_labels, test_size=0.2, random_state=42)
# Initialize the OneVsRestClassifier with LogisticRegression
ovr_classifier = OneVsRestClassifier(LogisticRegression(solver='lbfgs', max_iter=1000))

# Fit the model on the training data
ovr_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = ovr_classifier.predict(X_test)

# Calculate the F1 score for each class and average
f1_score_macro = f1_score(y_test, y_pred, average='macro')
f1_score_micro = f1_score(y_test, y_pred, average='micro')

f1_score_macro, f1_score_micro

(0.04535735735735736, 0.45903233649404324)

In [ ]:
import torch
import numpy as np
import pandas as pd
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision.models import resnet50
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score

# 데이터 로드
embeddings = np.load('/home/minhyekj/xraydata/chest-xray14_embeddings.npy')
data_entry = pd.read_csv('/home/minhyekj/xraydata/Data_Entry_2017.csv')
with open('/home/minhyekj/xraydata/selected_png_list.txt', 'r') as file:
    selected_images = file.read().splitlines()

# 이미지 파일 이름 추출
selected_image_filenames = [path.split('/')[-1] for path in selected_images]

# 레이블 매핑
image_label_map = dict(zip(data_entry['Image Index'], data_entry['Finding Labels']))
selected_labels = [image_label_map[filename] for filename in selected_image_filenames if filename in image_label_map]

# 레이블 이진화
mlb = MultiLabelBinarizer()
binary_labels = mlb.fit_transform([label.split('|') for label in selected_labels])

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(embeddings, binary_labels, test_size=0.2, random_state=42)

# 데이터 로더 설정
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ResNet50 모델 설정
class ResNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ResNetClassifier, self).__init__()
        base_model = resnet50(pretrained=True)
        num_features = base_model.fc.in_features
        base_model.fc = nn.Linear(num_features, num_classes)
        self.base_model = base_model

    def forward(self, x):
        return self.base_model(x)

model = ResNetClassifier(num_classes=binary_labels.shape[1])
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 훈련 함수
def train_model(model, train_loader, criterion, optimizer):
    model.train()
    total_loss = 0
    for data, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

# 모델 평가 함수
def evaluate_model(model, test_loader):
    model.eval()
    all_preds, all_targets = [], []
    with torch.no_grad():
        for data, targets in test_loader:
            outputs = model(data)
            predictions = torch.sigmoid(outputs)
            all_preds.append(predictions.cpu().numpy())
            all_targets.append(targets.cpu().numpy())
    all_preds = np.vstack(all_preds)
    all_targets = np.vstack(all_targets)
    return f1_score(all_targets, all_preds > 0.5, average='macro')

# 모델 훈련 및 평가
train_model(model, train_loader, criterion, optimizer)
f1 = evaluate_model(model, test_loader)
print("F1 Score on Test Set:", f1)
